# Lesson 4: Graph Convolutional Networks (GCNs)

In this notebook, we will:
1. Implement GCN layer from scratch
2. Use PyTorch Geometric's built-in GCNConv
3. Apply GCN to node classification on real datasets
4. Visualize learned embeddings
5. Analyze layer-by-layer behavior
6. Tune hyperparameters
7. Solve practical exercises

## Setup and Imports

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.datasets import Planetoid
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Check device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## Part 1: Understanding Graph Matrices

### Creating a Simple Graph

In [ ]:
# Create a simple graph
edge_list = torch.tensor([
    [0, 1], [1, 0], [1, 2], [2, 1],
    [0, 3], [3, 0], [1, 4], [4, 1],
    [2, 5], [5, 2], [3, 4], [4, 3],
    [4, 5], [5, 4],
], dtype=torch.long).t().contiguous()

n_nodes = 6
n_features = 3
x = torch.randn(n_nodes, n_features)
graph_data = Data(x=x, edge_index=edge_list)

print(f"Number of nodes: {graph_data.num_nodes}")
print(f"Number of edges: {graph_data.num_edges}")
print(f"Node features shape: {graph_data.x.shape}")

### Computing Graph Matrices

In [ ]:
def edge_index_to_adjacency(edge_index, num_nodes):
    A = torch.zeros(num_nodes, num_nodes)
    A[edge_index[0], edge_index[1]] = 1
    return A

def compute_degree_matrix(A):
    degree = A.sum(dim=1)
    D = torch.diag(degree)
    return D

def compute_laplacian(A):
    D = compute_degree_matrix(A)
    L = D - A
    return L

# Compute matrices
A = edge_index_to_adjacency(graph_data.edge_index, graph_data.num_nodes)
D = compute_degree_matrix(A)
L = compute_laplacian(A)

print("Adjacency Matrix A:")
print(A.int())
print("\nDegree Matrix D:")
print(D.int())
print("\nLaplacian Matrix L = D - A:")
print(L.int())

### Computing Normalized Adjacency

In [ ]:
def compute_normalized_adjacency(edge_index, num_nodes, add_self_loops=True):
    A = edge_index_to_adjacency(edge_index, num_nodes)
    if add_self_loops:
        A = A + torch.eye(num_nodes)
    D = compute_degree_matrix(A)
    D_inv_sqrt = torch.diag(1.0 / torch.sqrt(D.diag()))
    A_norm = D_inv_sqrt @ A @ D_inv_sqrt
    return A_norm

A_norm = compute_normalized_adjacency(graph_data.edge_index, graph_data.num_nodes)
print("Normalized Adjacency Matrix:")
print(A_norm)
print("\nRow sums:")
print(A_norm.sum(dim=1))

## Part 2: Implementing GCN Layer from Scratch

### GCN Layer Implementation

In [ ]:
class GCNLayerManual(nn.Module):
    def __init__(self, in_features, out_features, bias=True):
        super(GCNLayerManual, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = nn.Parameter(torch.Tensor(in_features, out_features))
        if bias:
            self.bias = nn.Parameter(torch.Tensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()
    
    def reset_parameters(self):
        nn.init.xavier_uniform_(self.weight)
        if self.bias is not None:
            nn.init.zeros_(self.bias)
    
    def forward(self, x, edge_index):
        num_nodes = x.size(0)
        A_norm = compute_normalized_adjacency(edge_index, num_nodes, add_self_loops=True)
        A_norm = A_norm.to(x.device)
        out = A_norm @ x @ self.weight
        if self.bias is not None:
            out = out + self.bias
        return out

# Test the layer
gcn_layer = GCNLayerManual(n_features, 8, bias=True)
output = gcn_layer(x, graph_data.edge_index)
print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")

### Complete GCN Model from Scratch

In [ ]:
class GCNManual(nn.Module):
    def __init__(self, in_features, hidden_features, out_features, num_layers=2, dropout=0.5):
        super(GCNManual, self).__init__()
        self.num_layers = num_layers
        self.dropout_prob = dropout
        self.layers = nn.ModuleList()
        
        self.layers.append(GCNLayerManual(in_features, hidden_features))
        for _ in range(num_layers - 2):
            self.layers.append(GCNLayerManual(hidden_features, hidden_features))
        self.layers.append(GCNLayerManual(hidden_features, out_features))
    
    def forward(self, x, edge_index):
        for i, layer in enumerate(self.layers):
            x = layer(x, edge_index)
            if i < len(self.layers) - 1:
                x = F.relu(x)
                x = F.dropout(x, p=self.dropout_prob, training=self.training)
        return x

# Test the model
model = GCNManual(n_features, 32, 4, num_layers=2, dropout=0.5)
output = model(x, graph_data.edge_index)
print(f"Model output shape: {output.shape}")
print(f"Total parameters: {sum(p.numel() for p in model.parameters()):,}")

## Part 3: Using PyTorch Geometric GCNConv

### PyTorch Geometric Implementation

In [ ]:
class GCN(nn.Module):
    def __init__(self, in_features, hidden_features, out_features, num_layers=2, dropout=0.5):
        super(GCN, self).__init__()
        self.num_layers = num_layers
        self.dropout_prob = dropout
        self.layers = nn.ModuleList()
        
        self.layers.append(GCNConv(in_features, hidden_features))
        for _ in range(num_layers - 2):
            self.layers.append(GCNConv(hidden_features, hidden_features))
        self.layers.append(GCNConv(hidden_features, out_features))
    
    def forward(self, x, edge_index):
        for i, layer in enumerate(self.layers):
            x = layer(x, edge_index)
            if i < len(self.layers) - 1:
                x = F.relu(x)
                x = F.dropout(x, p=self.dropout_prob, training=self.training)
        return x

# Test the model
model = GCN(n_features, 32, 4, num_layers=2, dropout=0.5)
output = model(x, graph_data.edge_index)
print(f"Output shape: {output.shape}")
print(f"Total parameters: {sum(p.numel() for p in model.parameters()):,}")

## Part 4: Node Classification on Real Dataset

### Load Cora Dataset

In [ ]:
# Load Cora dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0].to(device)

print(f"Dataset: {dataset}")
print(f"Number of nodes: {data.num_nodes}")
print(f"Number of edges: {data.num_edges}")
print(f"Number of features: {data.num_node_features}")
print(f"Number of classes: {dataset.num_classes}")
print(f"Train/Val/Test: {data.train_mask.sum()}/{data.val_mask.sum()}/{data.test_mask.sum()}")

### Training Loop

In [ ]:
def train(model, data, optimizer, criterion):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

@torch.no_grad()
def evaluate(model, data, criterion, mask):
    model.eval()
    out = model(data.x, data.edge_index)
    loss = criterion(out[mask], data.y[mask])
    pred = out[mask].argmax(dim=1)
    acc = (pred == data.y[mask]).sum().item() / mask.sum().item()
    return loss.item(), acc

print("Training and evaluation functions ready!")

### Full Training

In [ ]:
# Initialize model
model = GCN(
    in_features=data.num_node_features,
    hidden_features=64,
    out_features=dataset.num_classes,
    num_layers=2,
    dropout=0.5
).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

# Training
epochs = 200
train_losses = []
val_losses = []
val_accs = []
best_val_acc = 0
patience = 20
patience_counter = 0

print(f"Training GCN for {epochs} epochs...")

for epoch in range(epochs):
    train_loss = train(model, data, optimizer, criterion)
    val_loss, val_acc = evaluate(model, data, criterion, data.val_mask)
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    val_accs.append(val_acc)
    
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        patience_counter = 0
        best_state = model.state_dict()
    else:
        patience_counter += 1
    
    if patience_counter >= patience:
        print(f"Early stopping at epoch {epoch}")
        model.load_state_dict(best_state)
        break
    
    if (epoch + 1) % 20 == 0:
        print(f"Epoch {epoch+1:3d} | Train Loss: {train_loss:.4f} | Val Acc: {val_acc:.4f}")

print("Training completed!")

### Evaluation on Test Set

In [ ]:
# Evaluate on test set
test_loss, test_acc = evaluate(model, data, criterion, data.test_mask)

print(f"\n{'='*50}")
print(f"Final Results on Test Set:")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")
print(f"{'='*50}")

### Training Curves

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

axes[0].plot(train_losses, label='Train Loss', linewidth=2)
axes[0].plot(val_losses, label='Val Loss', linewidth=2)
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Loss', fontsize=12)
axes[0].set_title('Training and Validation Loss', fontsize=14)
axes[0].legend(fontsize=10)
axes[0].grid(alpha=0.3)

axes[1].plot(val_accs, label='Val Accuracy', linewidth=2, color='green')
axes[1].axhline(y=test_acc, color='red', linestyle='--', linewidth=2, label=f'Test Acc: {test_acc:.4f}')
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Accuracy', fontsize=12)
axes[1].set_title('Validation Accuracy Over Training', fontsize=14)
axes[1].legend(fontsize=10)
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## Part 5: Visualization of Learned Embeddings

### Extract Hidden Layer Embeddings

In [ ]:
def get_embeddings(model, data, layer_idx=-2):
    model.eval()
    x = data.x
    edge_index = data.edge_index
    
    with torch.no_grad():
        for i, layer in enumerate(model.layers):
            x = layer(x, edge_index)
            if i < len(model.layers) - 1:
                x = F.relu(x)
            if i == layer_idx:
                return x
    return x

embeddings_hidden = get_embeddings(model, data, layer_idx=0)
print(f"Hidden embeddings shape: {embeddings_hidden.shape}")

### t-SNE Visualization

In [ ]:
print("Computing t-SNE reduction...")
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
embeddings_tsne = tsne.fit_transform(embeddings_hidden.cpu().numpy())

plt.figure(figsize=(10, 8))
scatter = plt.scatter(
    embeddings_tsne[:, 0],
    embeddings_tsne[:, 1],
    c=data.y.cpu().numpy(),
    cmap='tab10',
    s=50,
    alpha=0.7,
    edgecolors='k',
    linewidth=0.5
)
plt.title('t-SNE of Hidden Layer Embeddings (Colored by Class)', fontsize=14)
plt.xlabel('t-SNE 1', fontsize=12)
plt.ylabel('t-SNE 2', fontsize=12)
plt.colorbar(scatter, label='Class')
plt.grid(alpha=0.3)
plt.show()

### PCA Visualization

In [ ]:
pca = PCA(n_components=2)
embeddings_pca = pca.fit_transform(embeddings_hidden.cpu().numpy())

print(f"Explained variance: {pca.explained_variance_ratio_}")

plt.figure(figsize=(10, 8))
scatter = plt.scatter(
    embeddings_pca[:, 0],
    embeddings_pca[:, 1],
    c=data.y.cpu().numpy(),
    cmap='tab10',
    s=50,
    alpha=0.7,
    edgecolors='k',
    linewidth=0.5
)
plt.title('PCA of Hidden Layer Embeddings', fontsize=14)
plt.xlabel(f"PC1 ({pca.explained_variance_ratio_[0]:.1%})", fontsize=12)
plt.ylabel(f"PC2 ({pca.explained_variance_ratio_[1]:.1%})", fontsize=12)
plt.colorbar(scatter, label='Class')
plt.grid(alpha=0.3)
plt.show()

## Part 6: Hyperparameter Tuning

### Experiment 1: Number of Layers

In [ ]:
print("Tuning: Number of Layers")
print("="*50)

results_layers = {}

for num_layers in [1, 2, 3, 4]:
    print(f"\nTraining GCN with {num_layers} layers...")
    
    model = GCN(
        in_features=data.num_node_features,
        hidden_features=64,
        out_features=dataset.num_classes,
        num_layers=num_layers,
        dropout=0.5
    ).to(device)
    
    optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    criterion = nn.CrossEntropyLoss()
    
    best_val_acc = 0
    test_acc = 0
    
    for epoch in range(200):
        train_loss = train(model, data, optimizer, criterion)
        val_loss, val_acc = evaluate(model, data, criterion, data.val_mask)
        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            test_loss, test_acc = evaluate(model, data, criterion, data.test_mask)
    
    results_layers[num_layers] = {'val_acc': best_val_acc, 'test_acc': test_acc}
    print(f"  Val Acc: {best_val_acc:.4f}, Test Acc: {test_acc:.4f}")

print("\nResults Summary:")
for num_layers, results in results_layers.items():
    print(f"  {num_layers} layers: Test Acc = {results['test_acc']:.4f}")

### Experiment 2: Dropout Rate

In [ ]:
print("\nTuning: Dropout Rate")
print("="*50)

results_dropout = {}

for dropout_rate in [0.0, 0.2, 0.5, 0.7]:
    print(f"\nTraining with dropout={dropout_rate}...")
    
    model = GCN(
        in_features=data.num_node_features,
        hidden_features=64,
        out_features=dataset.num_classes,
        num_layers=2,
        dropout=dropout_rate
    ).to(device)
    
    optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    criterion = nn.CrossEntropyLoss()
    
    best_val_acc = 0
    test_acc = 0
    
    for epoch in range(200):
        train_loss = train(model, data, optimizer, criterion)
        val_loss, val_acc = evaluate(model, data, criterion, data.val_mask)
        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            test_loss, test_acc = evaluate(model, data, criterion, data.test_mask)
    
    results_dropout[dropout_rate] = test_acc
    print(f"  Test Acc: {test_acc:.4f}")

print("\nResults Summary:")
for dropout_rate, test_acc in results_dropout.items():
    print(f"  dropout={dropout_rate}: Test Acc = {test_acc:.4f}")

### Visualization of Results

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Number of layers
layers = list(results_layers.keys())
test_accs_layers = [results_layers[l]['test_acc'] for l in layers]
axes[0].plot(layers, test_accs_layers, marker='o', linewidth=2, markersize=10)
axes[0].set_xlabel('Number of Layers', fontsize=12)
axes[0].set_ylabel('Test Accuracy', fontsize=12)
axes[0].set_title('Effect of Network Depth', fontsize=14)
axes[0].grid(alpha=0.3)
axes[0].set_xticks(layers)

# Dropout rate
dropouts = list(results_dropout.keys())
test_accs_dropout = [results_dropout[d] for d in dropouts]
axes[1].plot(dropouts, test_accs_dropout, marker='s', linewidth=2, markersize=10)
axes[1].set_xlabel('Dropout Rate', fontsize=12)
axes[1].set_ylabel('Test Accuracy', fontsize=12)
axes[1].set_title('Effect of Dropout', fontsize=14)
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## Exercises

Try these to deepen your understanding!

### Exercise 1: Add Residual Connections

Modify GCN to include skip connections: h_new = h_old + f(h_old)

In [ ]:
# TODO: Implement GCN with residual connections
print("Exercise 1: Add residual connections to prevent over-smoothing")
print("Hint: When dimensions match, add skip connection in forward()")

### Exercise 2: Try Other Datasets

Test on Citeseer and Pubmed datasets

In [ ]:
# TODO: Load and train on Citeseer
print("Exercise 2: Try Citeseer dataset")
print("from torch_geometric.datasets import Planetoid")
print("dataset = Planetoid(root='/tmp/Citeseer', name='Citeseer')")

### Exercise 3: Batch Normalization

Add batch normalization to layers

In [ ]:
# TODO: Add nn.BatchNorm1d to GCN layers
print("Exercise 3: Add batch normalization")
print("Use nn.BatchNorm1d between layers")

## Summary

We covered:
1. GCN implementation from scratch and using PyTorch Geometric
2. Semi-supervised node classification
3. Visualization of learned embeddings
4. Hyperparameter optimization
5. Architecture experiments

Key insights:
- GCNs aggregate information from neighbors via normalized adjacency
- Semi-supervised learning leverages graph structure
- 2-4 layers typically works best (over-smoothing)
- Proper regularization is crucial